In [1]:
import numpy as np
import os
import time
import scipy.io as sio
import pandas as pd
import time
import pickle

np.random.seed(0)

def windows(data, size):
    start = 0
    while ((start+size) < data.shape[0]):
        yield int(start), int(start + size)
        start += size
        
def data_1Dto2D(data, Y=9, X=9):
    data_2D = np.zeros([Y, X])
    data_2D[0] = (0,0,0,data[0],0,data[16],0,0,0)
    data_2D[1] = (0,0,0,data[1],0,data[17],0,0,0)
    data_2D[2] = (data[3],0,data[2],0,data[18],0,data[19],0,data[20])
    data_2D[3] = (0,data[4],0,data[5],0,data[22],0,data[21],0)
    data_2D[4] = (data[7],0,data[6],0,data[23],0,data[24],0,data[25])
    data_2D[5] = (0,data[8],0,data[9],0,data[27],0,data[26],0)
    data_2D[6] = (data[11],0,data[10],0,data[15],0,data[28],0,data[29])
    data_2D[7] = (0,0,0,data[12],0,data[30],0,0,0)
    data_2D[8] = (0,0,0,data[13],data[14],data[31],0,0,0)
    # return shape:9*9
    return data_2D


def dataset_1Dto2D(dataset_1D):
    dataset_2D = np.zeros([dataset_1D.shape[0],9,9])
    for i in range(dataset_1D.shape[0]):
        dataset_2D[i] = data_1Dto2D(dataset_1D[i])
    # return shape: m*9*9
    return dataset_2D

def apply_mixup(dataset_file,window_size,label,yes_or_not,fam_label): # initial empty label arrays
    
    print("Processing",dataset_file,'----------------')
    
#     file=dataset_dir+record_list[0]
#     label='valence'
    data_file_in = sio.loadmat(dataset_file)
#     print(data_file_in['data'].shape)
#     print(data_file_in['data'].transpose(0,2,1).shape)
    data_in = data_file_in["data"].transpose(0,2,1)
    if label=="arousal":
        label=1
    elif label=="valence":
        label=0
    label_in=fam_label>3
#     label_in= data_file_in["labels"][:,label]>5
#     print(label_in.shape,label_in[1])
    # label_in
    label_inter	= np.empty([0]) # initial empty data arrays
    data_inter_cnn	= np.empty([0,window_size, 9, 9])
    data_inter_rnn	= np.empty([0, window_size, 32])
    trials = data_in.shape[0]
#     print(data_inter_cnn.shape,data_inter_rnn.shape,trials)
    # Data pre-processing
    # for trial in range(0,trials):
    # data_inter_cnn[0]
    # print(data_in)
#     yes_or_not='yes'
    for trial in range(0,trials):#trials
        if yes_or_not=="yes":
            base_signal = (data_in[trial,0:128,0:32]+data_in[trial,128:256,0:32]+data_in[trial,256:384,0:32])/3
    #         print(base_signal.shape)
    #         base_signal/=3
    #         print('+++++',base_signal.shape)

        else:
            base_signal = 0
        data = data_in[trial,384:8064,0:32]
    #     print(data.shape)
        # compute the deviation between baseline signals and experimental signals
        for i in range(0,60):
            data[i*128:(i+1)*128,0:32]=data[i*128:(i+1)*128,0:32]-base_signal
        label_index = trial
    #     print(data.shape)
#         label_index = trial
        #read data and label
    #     data = norm_dataset(data)

        norm_dataset_1D = np.zeros([data.shape[0], 32])
        for i in range(data.shape[0]):
    #         norm_dataset_1D[i] = feature_normalize(data[i])
            data_i=data[i]
            mean = data_i[data_i.nonzero()].mean()
            sigma = data_i[data_i. nonzero ()].std()
            data_normalized = data_i
            data_normalized[data_normalized.nonzero()] = (data_normalized[data_normalized.nonzero()] - mean)/sigma

            norm_dataset_1D[i]=data_normalized

        # 8064 - 7680 = 384 = 128*3 (for 3 secs & 128 hertz each)
    #     print('=======')
    #     print(norm_dataset_1D.shape)


    #     data, label = segment_signal_without_transition(data, label_in,label_index,window_size)


        for (start, end) in windows(data, window_size):
            # print(data.shape)
            if((len(data[start:end]) == window_size)):
                if(start == 0):
                    segments = data[start:end]
                    segments = np.vstack([segments, data[start:end]])

                    labels = np.array(label_in[label_index])
                    labels = np.append(labels, np.array(label_in[label_index]))
                else:
                    segments = np.vstack([segments, data[start:end]])
                    labels = np.append(labels, np.array(label_in[label_index])) # labels = np.append(labels, stats.mode(label[start:end])[0][0])
    #     return segments, labels
        data,label=segments,labels

    #     print(trial,'trial ',data.shape,label.shape)

        # cnn data process
    #     print('++++++++')
        data_cnn    = dataset_1Dto2D(data)
    #     print(data_cnn.shape)
        data_cnn    = data_cnn.reshape(int(data_cnn.shape[0]/window_size), window_size, 9, 9)
    #     print(data_cnn.shape)
        # rnn data process
        data_rnn    = data.reshape(int(data.shape[0]/window_size), window_size, 32)
    #     print(data_rnn.shape)
        # append new data and label
        data_inter_cnn  = np.vstack([data_inter_cnn, data_cnn])
        data_inter_rnn  = np.vstack([data_inter_rnn, data_rnn])
        label_inter = np.append(label_inter, label)
    #     print(data_inter_cnn.shape)
    #     print(data_inter_rnn.shape)
    #     print(label_inter.shape)

    # shuffle data
    index = np.array(range(0, len(label_inter)))
    np.random.shuffle(index)
    shuffled_data_cnn= data_inter_cnn[index]
    shuffled_data_rnn= data_inter_rnn[index]
    shuffled_label= label_inter[index]
#     print(shuffled_data_cnn.shape) # what is 120 for 120,128,9,9
#     print(shuffled_data_rnn.shape)
#     print(shuffled_label.shape)
#     print(len(record_list))
    return shuffled_data_cnn ,shuffled_data_rnn,shuffled_label,record_list


In [2]:
label_classes_list=['familiarity']
suffixes_list=['no','yes']

In [3]:
#label_in= data_file_in["labels"][:,label]>5

In [4]:
begin=time.time()
count=1
for label_class in label_classes_list:
    for suffix in suffixes_list:
        print('----------------------------------------------------------------')
        print('label_class = ',label_class,'suffix = ',suffix)
        begin1 = time.time()
        # print("time begin:",time.localtime())
        dataset_dir='deap_matlab_files/'
        
        
        
        
#         label_class='arousal'
#         suffix='yes'
        window_size=128
        record_list=[i for i in os.listdir(dataset_dir) if 
                     os.path.isfile(os.path.join(dataset_dir,i))]
        # record_list
        output_dir=   "./deap_shuffled_data/"
        output_dir = output_dir+suffix+"_"+label_class+"/"
        if os.path.isdir(output_dir)==False:
                os.makedirs(output_dir)

        for i in record_list:
            begin2=time.time()
            
            file = os.path.join(dataset_dir,i)
            
            
            snf=int(file[-6:-4])
            pkllist=os.listdir('familiarity_pkl/')
            if('sub_'+str(snf)+'.pkl' in pkllist):
                fam_label=pickle.load(open('familiarity_pkl/sub_'+str(snf)+'.pkl','rb'))
    #             print(snf,fam_label.shape)
            else:
                continue
            
            
            
            shuffled_cnn_data,shuffled_rnn_data,shuffled_label,record = apply_mixup(file, window_size,label_class,suffix,fam_label)
            output_data_cnn = output_dir+i+"_win_"+str(window_size)+"_cnn_dataset.pkl"
            output_data_rnn = output_dir+i+"_win_"+str(window_size)+"_rnn_dataset.pkl"
            output_label= output_dir+i+"_win_"+str(window_size)+"_labels.pkl"

            with open(output_data_cnn, "wb") as fp:
                pickle.dump( shuffled_cnn_data,fp, protocol=4)
            with open( output_data_rnn, "wb") as fp:
                pickle.dump(shuffled_rnn_data, fp, protocol=4)
            with open(output_label, "wb") as fp:
                pickle.dump(shuffled_label, fp)
                
            end2=time.time()
            print('record ',count,' time -> ',(end2-begin2))
            count+=1
        
        end1 = time.time()
        #     print("end time:",time.localtime())
        # https://stackoverflow.com/questions/23582489/python-pickle-protocol-choice
        print('combination time -> ',(end1-begin1))
        print('----------------------------------------------------------------')

end=time.time()
print('total time -> ',(end-begin))

----------------------------------------------------------------
label_class =  familiarity suffix =  no
Processing deap_matlab_files/s01.mat ----------------
record  1  time ->  38.621333360672
Processing deap_matlab_files/s03.mat ----------------
record  2  time ->  41.47250723838806
Processing deap_matlab_files/s04.mat ----------------
record  3  time ->  21.00308346748352
Processing deap_matlab_files/s05.mat ----------------
record  4  time ->  15.966233015060425
Processing deap_matlab_files/s06.mat ----------------
record  5  time ->  16.892324686050415
Processing deap_matlab_files/s07.mat ----------------
record  6  time ->  17.502416372299194
Processing deap_matlab_files/s08.mat ----------------
record  7  time ->  18.63066029548645
Processing deap_matlab_files/s09.mat ----------------
record  8  time ->  18.17702054977417
Processing deap_matlab_files/s10.mat ----------------
record  9  time ->  17.393477201461792
Processing deap_matlab_files/s11.mat ----------------
record  10 

In [5]:
1008.2821416854858/60

16.804702361424763

In [4]:
3778.8399555683136/60.0

62.98066592613856

In [5]:
 3561.819000005722/60

59.36365000009537

In [23]:
for label_class in label_classes_list:
    for suffix in suffixes_list:
        dataset_dir='deap_matlab_files/'
        record_list=[i for i in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir,i))]
        #         record_list
        output_dir=   "./deap_shuffled_data2/"
        output_dir = output_dir+suffix+"_"+label_class+"/"
        if os.path.isdir(output_dir)==False:
                os.makedirs(output_dir)

        for i in record_list:

            file = os.path.join(dataset_dir,i)
            snf=int(file[-6:-4])
            pkllist=os.listdir('familiarity_pkl/')
            if('sub_'+str(snf)+'.pkl' in pkllist):
                fam_label=pickle.load(open('familiarity_pkl/sub_'+str(snf)+'.pkl','rb'))
                print(snf,fam_label.shape)
#             shuffled_cnn_data,shuffled_rnn_data,shuffled_label,record = apply_mixup(file, window_size,label_class,suffix)

1 (40,)
3 (40,)
4 (40,)
5 (40,)
6 (40,)
7 (40,)
8 (40,)
9 (40,)
10 (40,)
11 (40,)
12 (40,)
13 (40,)
14 (40,)
16 (40,)
17 (40,)
18 (40,)
19 (40,)
20 (40,)
21 (40,)
22 (40,)
24 (40,)
25 (40,)
26 (40,)
27 (40,)
28 (40,)
29 (40,)
30 (40,)
31 (40,)
32 (40,)
1 (40,)
3 (40,)
4 (40,)
5 (40,)
6 (40,)
7 (40,)
8 (40,)
9 (40,)
10 (40,)
11 (40,)
12 (40,)
13 (40,)
14 (40,)
16 (40,)
17 (40,)
18 (40,)
19 (40,)
20 (40,)
21 (40,)
22 (40,)
24 (40,)
25 (40,)
26 (40,)
27 (40,)
28 (40,)
29 (40,)
30 (40,)
31 (40,)
32 (40,)


In [22]:
str(9)

'9'

In [12]:
df=sio.loadmat('deap_matlab_files/s01.mat')

In [17]:
df['labels'][:,1].shape

(40,)